# Python Database Programming

-----

Previously, we were introduced to SQL and learned how to use SQL to create a schema, insert data into the new schema, how to query these data, and how to modify and delete these data and the schema itself. However, this was all done manually by using a database client tool, specifically the SQLite command line tool. A more efficient mechanisms is to enable automatic database interactions from within a Python program.

In this lesson, we explore how to connect to a [SQLite database][1] with a Python program. This will build on ideas presented in the previous two lessons. While some of what we do will be SQLite specific, most of what we do will be generic and easily [applied to other databases][2] for which a [Python database interface library][3] has been developed, which is for most major SQL based databases. Finally, we will explore how the Pandas library and the DataFrame can be used to simplify these steps.

-----
[1]: https://docs.python.org/3/library/sqlite3.html
[2]: https://www.python.org/dev/peps/pep-0249/
[3]: https://wiki.python.org/moin/DatabaseInterfaces

## Table of Contents


[Python and SQLite](#Python-and-SQLite)

- [Python: Database Connections](#Python:-Database-Connections)
- [Database Cursor](#Database-Cursor)
- [Executing SQL Commands](#Executing-SQL-Commands)
- [Parameterized Queries](#Parameterized-Queries)

[Pandas and SQL](#Pandas-and-SQL)

-----
[[Back to TOC]](#Table-of-Contents)

## Python and SQLite

To use the SQLite database from within a Python program, we simply need to import the sqlite3 module. Since SQLite is distributed with Python, there are no extra download or installation steps. If you wish to use a different database, you will first need to download the appropriate Python library for that database; and, second, install the library to the appropriate system path.

In the following code cell, we demonstrate how to use SQLite from within a Python program. The first step is to simply import the sqlite3 module. After this we can access the module to interact with a database. In the first example, we simply access several module attributes that specify the version information for both the SQLIte database and sqlite3 module we currently have installed.

-----

In [1]:
import sqlite3 as sql

print(f'SQLite library version: {sql.version}')
print(f'SQLite version: {sql.sqlite_version}')

SQLite library version: 2.6.0
SQLite version: 3.29.0


-----

Since the `sqlite3` module is part of the standard Python library, this import process should be painless. We also can take advantage of the fact that the SQLite database itself is embedded within the Python interpreter. As a result, we can easily view the built-in help information for either the sqlite3 module or for specific attributes, functions, or types that are in the sqlite3 module by using the Python `help()` function.

-----

In [2]:
# View built-in help for module
help(sql.connect)

Help on built-in function connect in module _sqlite3:

connect(...)
    connect(database[, timeout, detect_types, isolation_level,
            check_same_thread, factory, cached_statements, uri])
    
    Opens a connection to the SQLite database file *database*. You can use
    ":memory:" to open a database connection to a database that resides in
    RAM instead of on disk.



-----
[[Back to TOC]](#Table-of-Contents)

### Python: Database Connections

To work with a database from within a Python program, we follow a specific set of operations:

1. Establish a _connection_ to the database. With most database systems, this involves a [network connection][1] to a (possibly) remote database server. SQLite is an embedded database, however, so we, by default, simply need the database file to be opened by the SQLite library.

2. Obtain a _cursor_ from the database connection. A [database cursor][2] is an object that enables us to programmatically move through a database table in order to insert, delete, update, or select data. 

3. Execute SQL commands by using the database cursor. These commands can be SQL DDL commands where we create schema, or SQL DML commands where we insert, update, delete, or select data. The execution process can return information that we can programmatically use.

4. Commit the transactions. Database updates become permenant and visible to other database connections only after the transaction is committed.

5. Close the cursor and the connection. This action is necessary to release the resources allocated to the database cursor and connection.

We obtain a database connection by calling the `connect` built-in method in the sqlite3 library. The only required parameter is the name of the database, which is the pathname to the database file. For example, we could connect to the test database created in previous lesson by specifying the path, `testdb`. This method returns an instance of the [SQLite Connection][3] object. If `testdb` doesn't exist, the function will create an empty database with the name.

```python
conn = sql.connect("testdb")
```

We commit database transaction by calling `commit` method of the connection object.
```python
conn.commit()
```

A database connection needs to be closed to make sure the connection resources are properly released. 
```python
conn.close()
```

With context manager, we don't have to commit the database transactions explicitly. We do this by using the `with` statement. This will ensure that if the operations contained within the context code block are successful, that database transaction will be committed. If there is a problem within the context, or an exception is thrown, however, the transaction will instead be rolled back. As a result, the database will be maintained in a consistent state. Please note that the database connection won't be closed when exiting the context. We still need to close the connection explicitly to release the connection resource. All operations related to the connection should be in the context of the connection which is defined by proper indentation.

```python
with sql.connect("testdb") as conn:
    #db operations in the context of conn
#Close connection explicitly
conn.close()
```

-----

[1]: https://en.wikipedia.org/wiki/Database_connection
[2]: https://en.wikipedia.org/wiki/Cursor_(databases)
[3]: https://docs.python.org/3/library/sqlite3.html#connection-objects

[[Back to TOC]](#Table-of-Contents)

### Database Cursor

To interact with a database, we need a cursor. We can use an implicit cursor by calling execute methods directly on the connection object. The recommended approach, however, is to always be explicit, and in this case that means we use an explicit cursor. The following code sample demonstrates how to acquire a cursor from a database connection, `con`. The method will return an instance of the [SQLite Cursor][sc] object.

```python
cur = conn.cursor()    
```

To close a cursor, we call `close` method of the cursor object.
```python
cur.close()
```



-----
[sc]: https://docs.python.org/3/library/sqlite3.html#cursor-objects

[[Back to TOC]](#Table-of-Contents)

### Executing SQL Commands

Once we have a cursor object, we can begin to execute SQL commands. There are several methods that we can use, depending on our specific needs: 

- `execute()` executes a single SQL statement. The SQL statement can be parameterized, in which case the replacement values also must be passed to the execute method.

- `executemany()` executes a SQL statement multiple times. The SQL statement, which is the first argument is parameterized and with each new invocation the parameters are replaced by the values contained in the second argument. This can be useful for inserting data into an existing table.

- `executescript()` executes multiple SQL statements simultaneously. A transaction commit is first performed, and then the SQL statements contained in the argument passed into the method are evaluated.

After we have executed one or more SQL query statements, our cursor object will enable us to _fetch_ the results. There are three different fetch operations:

- `fetchone()` returns the next row in the query result. The return data type is a single sequence containing the values in the row. If no data was returned, `None` is instead returned.

- `fetchmany()` returns the next set of rows in the query result. The return data type is a list containing the individual rows (which are sequences). A size parameter can be supplied to indicate how many rows should be returned.

- `fetchall()` returns all (remaining) rows of a query result. The return data type is a list, and if no more results remain, and empty list is returned.

In the next code cell, we create connection and cursor directly and run a query with the cursor. We need to close the cursor and the connection at the end to release the resourses. We don't need to commit then transaction since we only read from the database and there is no database updates to be committed. Then in the following code cell, we use context manager to do the same thing. 

-----

In [3]:
# Get connection
conn = sql.connect("testdb")
# Get cursor
cur = conn.cursor()
# Get sqlite version with a query
cur.execute('SELECT SQLITE_VERSION()')
# Fetch result
data = cur.fetchone()
print(f'SQLite version: {data[0]}')

# Close cursor and connection
cur.close()
conn.close()

SQLite version: 3.29.0


In [4]:
# Open a database connection
with sql.connect("testdb") as conn:

    # Now we obtain our cursor and execute a simple query.
    cur = conn.cursor()
    cur.execute('SELECT SQLITE_VERSION()')
    
    # Our simple query has one return value, so we only need to fetch one
    data = cur.fetchone()
    
    # Output the information
    print(f'SQLite version: {data[0]}')
# Close cursor and connection
cur.close()
conn.close()


SQLite version: 3.29.0


-----
[[Back to TOC]](#Table-of-Contents)

### Parameterized Queries

In the previous example, we 
1. established a database connection,
2. obtained our cursor,
3. executed a simple query to obtain the SQLite database version information, and
4. fetched the return value before displaying the result.

While trivial, this example does demonstrate how to perform the basic steps of working with a SQLite database. In the next example, however, we return to our Bigdog's Surf Shop example to programmatically create and populate a schema before issuing a query. In this case, we use a predefined SQL string to create our table, and use a tuple sequence to pass our data into the `executemany()` function. As shown in the example, the parameters are indicated in the SQL INSERT statement as question mark character format `?`. A second technique exists in which the parameters are explicitly named; this will be demonstrated later. The `executemany()` method replaces the parameters by the corresponding values from the `items` sequence, continuing until the sequence of items has been exhausted.

After the schema has been created and populated, we next issue a SQL query. In this case, we use the fact that the `execute()` method can be used as an iterator to iteratively process each row returned from our query. The `row` object is a sequence, which easily allows us to extract the desired columns.

-----

In [5]:
# Tuple containing data values to insert into our database
items = ((10,19.95,104,'2018-03-31','Hooded sweatshirt'), 
         (11,99.99,104, '2018-03-29','Beach umbrella'),
         (12,0.99,104,'2018-02-28', None))

# Open a database connection, here we use an in memory DB

with sql.connect("testdb") as conn:

    # Now we obtain our cursor
    cur = conn.cursor()
    
    # Insert some values to myProducts
    cur.executemany("INSERT INTO myProducts VALUES(?, ?, ?, ?, ?)", items)

    print("-"*50)
    print("All products:")
    print("-"*50)
    # Select rows and iterate through them
    for row in cur.execute('SELECT * FROM myProducts'):
        print(f"| {row[0]} | {row[1]} | {row[2]} | {row[3]} | {row[4]} |")
        
    print("-"*50)
    print("Beach umbrellas:")
    print("-"*50)
    # Select by description
    desc = 'Beach umbrella'
    for row in cur.execute(f'SELECT * FROM myProducts WHERE description="{desc}"'):
        print(f"| {row[0]} | {row[1]} | {row[2]} | {row[3]} | {row[4]} |")
        
    # Delete some rows
    cur.execute("DELETE FROM myProducts WHERE itemNumber >= 10")
# Close cursor and connection
cur.close()
conn.close()


--------------------------------------------------
All products:
--------------------------------------------------
| 6 | 25.9 | 103 | 2019-04-01 | Flip-flop |
| 7 | 34.95 | 103 | 2015-01-24 | Open-toed sandal |
| 10 | 19.95 | 104 | 2018-03-31 | Hooded sweatshirt |
| 11 | 99.99 | 104 | 2018-03-29 | Beach umbrella |
| 12 | 0.99 | 104 | 2018-02-28 | None |
--------------------------------------------------
Beach umbrellas:
--------------------------------------------------
| 11 | 99.99 | 104 | 2018-03-29 | Beach umbrella |


-----

## Pandas and SQL
Pandas provides built-in support for executing a SQL query and retrieving the result as a DataFrame. This is demonstrated in the next code cell, where we execute a SQL query on the test database. 

The Pandas method to execute a SQL statement is `read_sql`, and mimics in appearance other Panda methods for _reading_ data into a Pandas DataFrame. In this case, the method takes our SQL statement, a database connection, and an optional parameter, `index_col` that we can use to specify which column in our result should be treated as an index column for the new DataFrame. Pandas will supply an auto-incrementing column if no column is explicitly supplied. We demonstrate with and without setting `index_col` in below example.

-----

In [6]:
import pandas as pd
# Extract data into a Pandas DataFrame
query = '''
SELECT itemNumber, price, stockDate, description
FROM myProducts;'''

# Use SQL Context
with sql.connect("testdb") as conn:
    #withouth setting index_col
    data1 = pd.read_sql(query, conn)
    #set index_col
    data2 = pd.read_sql(query, conn, index_col='itemNumber')
    
# Close connection
conn.close()

data1.head()

,itemNumber,price,stockDate,description
0,6,25.90,2019-04-01,Flip-flop
1,7,34.95,2015-01-24,Open-toed sandal


In [7]:
data2.head()

,price,stockDate,description
itemNumber,,,
6,25.90,2019-04-01,Flip-flop
7,34.95,2015-01-24,Open-toed sandal


-----

This technique can be used for arbitrarily complex SQL statements. In the next code cell, we modify our SQL query to select products of a certain supplier from the testdb database. This time we don't specify index column so that the DataFrame will supply a default row index.

-----

In [8]:
# Find products by a certain supplier
supplier = 'Quiet Beach Industries'
query = f'''
SELECT p.itemNumber, p.price, p.stockDate, p.description, s.supplierName
FROM myProducts p, mySuppliers s
WHERE p.supplierNumber=s.supplierNumber
AND s.supplierName="{supplier}";'''

# Use SQL Context
with sql.connect("testdb") as conn:
    product_supplier = pd.read_sql(query, conn)
# Close connection
conn.close()    

# Display result
product_supplier

,itemNumber,price,stockDate,description,supplierName
0,6,25.90,2019-04-01,Flip-flop,Quiet Beach Industries
1,7,34.95,2015-01-24,Open-toed sandal,Quiet Beach Industries


-----

Pandas also simplifies the insertion of new data into a SQL database. For this, we can simply take an existing Pandas DataFrame and call the `to_sql()` method. This method requires at least two parameters, the name of the database table and the database connection. If the table does not exist, a new table will be created to match the DataFrame, including appropriate column names and data types. If the table exists, we need to set function parameter `if_exists` to determine what to do. Default value of `if_exists` is 'fail':
- fail: Raise a ValueError.
- replace: Drop the table before inserting new values.
- append: Insert new values to the existing table.  

Please check out the document of `to_sql()` for more details. In the following code cell, we will insert the DataFrame created from the above example to the new table. If the table already exists, `if_exists='replace'` makes sure the existing table will be dropped first. We pass an extra parameter to `to_sql()` function, `index=False`. This is to prevent creating a column in the database table for the index of the DataFrame.

-----

In [9]:
with sql.connect("testdb") as conn:
    product_supplier.to_sql("myProductSupplier", conn, if_exists='replace', index=False)
# Close connection
conn.close()

In [10]:
with sql.connect("testdb") as conn:
    data = pd.read_sql("SELECT * FROM myProductSupplier", conn)
# Close connection
conn.close()
data.head()

,itemNumber,price,stockDate,description,supplierName
0,6,25.90,2019-04-01,Flip-flop,Quiet Beach Industries
1,7,34.95,2015-01-24,Open-toed sandal,Quiet Beach Industries


## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. [Official][1] SQLite Python DB-API implementation
2. The [SQLIte Python Tutorial][2], is a bit old and somewhat out-of-date, but it provides useful perspective.
3. [PEP-249][3], which outlines the common database API in Python

-----

[1]: https://docs.python.org/3/library/sqlite3.html
[2]: http://zetcode.com/db/sqlitepythontutorial/
[3]: https://www.python.org/dev/peps/pep-0249/


**&copy; 2019: Gies College of Business at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode